In [3]:
import sys

from langchain.llms  import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

import pandas as pd
import os
import sys
import json

# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

['/home/llmuser', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/llmuser/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages']


In [4]:
# Make sure the model path is correct for your system!
local_llm = LlamaCpp(
    model_path="models/mixtral-8x7b-v0.1.Q5_K_M.gguf",
    repeat_penalty=1.1,
    temperature=0.0,
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=1024*10,
    top_p=1,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)


llama_model_loader: loaded meta data with 25 key-value pairs and 995 tensors from models/mixtral-8x7b-v0.1.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mixtral-8x7b-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.at

0:6:45


In [5]:
schema_linking_prompt = '''Table advisor, columns = [*,s_ID,i_ID]
Table classroom, columns = [*,building,room_number,capacity]
Table course, columns = [*,course_id,title,dept_name,credits]
Table department, columns = [*,dept_name,building,budget]
Table instructor, columns = [*,ID,name,dept_name,salary]
Table prereq, columns = [*,course_id,prereq_id]
Table section, columns = [*,course_id,sec_id,semester,year,building,room_number,time_slot_id]
Table student, columns = [*,ID,name,dept_name,tot_cred]
Table takes, columns = [*,ID,course_id,sec_id,semester,year,grade]
Table teaches, columns = [*,ID,course_id,sec_id,semester,year]
Table time_slot, columns = [*,time_slot_id,day,start_hr,start_min,end_hr,end_min]
Foreign_keys = [course.dept_name = department.dept_name,instructor.dept_name = department.dept_name,section.building = classroom.building,section.room_number = classroom.room_number,section.course_id = course.course_id,teaches.ID = instructor.ID,teaches.course_id = section.course_id,teaches.sec_id = section.sec_id,teaches.semester = section.semester,teaches.year = section.year,student.dept_name = department.dept_name,takes.ID = student.ID,takes.course_id = section.course_id,takes.sec_id = section.sec_id,takes.semester = section.semester,takes.year = section.year,advisor.s_ID = student.ID,advisor.i_ID = instructor.ID,prereq.prereq_id = course.course_id,prereq.course_id = course.course_id]
Q: "Find the buildings which have rooms with capacity more than 50."
A: Let’s think step by step. In the question "Find the buildings which have rooms with capacity more than 50.", we are asked:
"the buildings which have rooms" so we need table = [classroom] since we need the column = [classroom.capacity]
"rooms with capacity" so we need column = [classroom.building]
"rooms with capacity" so we need table = [classroom] since we need the column = [classroom.building]
Based on the tables, columns, and Foreign_keys. So the Tables_required are:
Tables_Required: ['classroom']

Table department, columns = [*,Department_ID,Name,Creation,Ranking,Budget_in_Billions,Num_Employees]
Table head, columns = [*,head_ID,name,born_state,age]
Table management, columns = [*,department_ID,head_ID,temporary_acting]
Foreign_keys = [management.head_ID = head.head_ID,management.department_ID = department.Department_ID]
Q: "How many heads of the departments are older than 56 ?"
A: Let’s think step by step. In the question "How many heads of the departments are older than 56 ?", we are asked:
"How many heads of the departments" so we need table = [head] since we need the column = [head.*]
"older" than 56 so we need table = [head] since we need the column = [head.age]
Based on the tables, columns. So the Tables_required are:
Tables_Required: ['head']

Table Country, columns = [*,id,name]
Table League, columns = [*,id,country_id,name]
Table Player, columns = [*,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight]
Table Player_Attributes, columns = [*,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes]
Table Team, columns = [*,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name]
Table Team_Attributes, columns = [*,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassing,chanceCreationPassingClass,chanceCreationCrossing,chanceCreationCrossingClass,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass]
Table sqlite_sequence, columns = [*,name,seq]
Foreign_keys = [Player_Attributes.player_api_id = Player.player_api_id,Player_Attributes.player_fifa_api_id = Player.player_fifa_api_id,League.country_id = Country.id,Team_Attributes.team_api_id = Team.team_api_id,Team_Attributes.team_fifa_api_id = Team.team_fifa_api_id]
Q: "List the names of all left-footed players who have overall rating between 85 and 90."
A: Let’s think step by step. In the question "List the names of all left-footed players who have overall rating between 85 and 90.", we are asked:
"names of all left-footed players" so we need table = [Player] since we need the column = [Player.player_name,Player_Attributes.preferred_foot]
"players who have overall rating" so we need table = [Player_Attributes] since we need column = [Player_Attributes.overall_rating]
Based on the tables, columns. So the Tables_required are:
Tables_Required: ['Player', 'Player_Attributes']
'''

In [6]:
DATASET_SCHEMA = '/home/llmuser/models/datasets/spider-2/tables.json'
DATASET = '/home/llmuser/models/datasets/spider-2/dev.json'
OUTPUT_FILE = '/home/llmuser/models/results/'

def load_data(DATASET):
    return pd.read_json(DATASET)

def local_generation(prompt):
    return local_llm(prompt)

def schema_linking_prompt_maker(test_sample_text,database):
  instruction = "# Find the schema_links for generating SQL queries for each question based on the database schema and Foreign keys.\n"
  fields = find_fields_MYSQL_like(database)
  foreign_keys = "Foreign_keys = " + find_foreign_keys_MYSQL_like(database) + '\n'
  prompt = instruction + schema_linking_prompt + fields + 'Q: "' + test_sample_text + """"\nA: Let’s think step by step."""
  #prompt = instruction + schema_linking_prompt + fields +foreign_keys+ 'Q: "' + test_sample_text + """"\nA: Let’s think step by step."""  
  return prompt

def find_fields_MYSQL_like(db_name):
  df = spider_schema[spider_schema['Database name'] == db_name]
  df = df.groupby(' Table Name')
  output = ""
  for name, group in df:
    output += "Table " +name+ ', columns = ['
    for index, row in group.iterrows():
      output += row[" Field Name"]+','
    output = output[:-1]
    output += "]\n"
  return output

def find_foreign_keys_MYSQL_like(db_name):
  df = spider_foreign[spider_foreign['Database name'] == db_name]
  output = "["
  for index, row in df.iterrows():
    output += row['First Table Name'] + '.' + row['First Table Foreign Key'] + " = " + row['Second Table Name'] + '.' + row['Second Table Foreign Key'] + ','
  output= output[:-1] + "]"
  return output

def creatiing_schema(DATASET_JSON):
    schema_df = pd.read_json(DATASET_JSON)
    schema_df = schema_df.drop(['column_names','table_names'], axis=1)
    schema = []
    f_keys = []
    p_keys = []
    for index, row in schema_df.iterrows():
        tables = row['table_names_original']
        col_names = row['column_names_original']
        col_types = row['column_types']
        foreign_keys = row['foreign_keys']
        primary_keys = row['primary_keys']
        for col, col_type in zip(col_names, col_types):
            index, col_name = col
            if index == -1:
                for table in tables:
                    schema.append([row['db_id'], table, '*', 'text'])
            else:
                schema.append([row['db_id'], tables[index], col_name, col_type])
        for primary_key in primary_keys:
            index, column = col_names[primary_key]
            p_keys.append([row['db_id'], tables[index], column])
        for foreign_key in foreign_keys:
            first, second = foreign_key
            first_index, first_column = col_names[first]
            second_index, second_column = col_names[second]
            f_keys.append([row['db_id'], tables[first_index], tables[second_index], first_column, second_column])
    spider_schema = pd.DataFrame(schema, columns=['Database name', ' Table Name', ' Field Name', ' Type'])
    spider_primary = pd.DataFrame(p_keys, columns=['Database name', 'Table Name', 'Primary Key'])
    spider_foreign = pd.DataFrame(f_keys,
                        columns=['Database name', 'First Table Name', 'Second Table Name', 'First Table Foreign Key',
                                 'Second Table Foreign Key'])
    return spider_schema,spider_primary,spider_foreign


def creatiing_schema(DATASET_JSON):
    schema_df = pd.read_json(DATASET_JSON)
    schema_df = schema_df.drop(['column_names','table_names'], axis=1)
    schema = []
    f_keys = []
    p_keys = []
    for index, row in schema_df.iterrows():
        tables = row['table_names_original']
        col_names = row['column_names_original']
        col_types = row['column_types']
        foreign_keys = row['foreign_keys']
        primary_keys = row['primary_keys']
        for col, col_type in zip(col_names, col_types):
            index, col_name = col
            if index == -1:
                for table in tables:
                    schema.append([row['db_id'], table, '*', 'text'])
            else:
                schema.append([row['db_id'], tables[index], col_name, col_type])
        for primary_key in primary_keys:
            index, column = col_names[primary_key]
            p_keys.append([row['db_id'], tables[index], column])
        for foreign_key in foreign_keys:
            first, second = foreign_key
            first_index, first_column = col_names[first]
            second_index, second_column = col_names[second]
            f_keys.append([row['db_id'], tables[first_index], tables[second_index], first_column, second_column])
    spider_schema = pd.DataFrame(schema, columns=['Database name', ' Table Name', ' Field Name', ' Type'])
    spider_primary = pd.DataFrame(p_keys, columns=['Database name', 'Table Name', 'Primary Key'])
    spider_foreign = pd.DataFrame(f_keys,
                        columns=['Database name', 'First Table Name', 'Second Table Name', 'First Table Foreign Key',
                                 'Second Table Foreign Key'])
    return spider_schema,spider_primary,spider_foreign

In [ ]:
if __name__ == '__main__':
    spider_schema,spider_primary,spider_foreign = creatiing_schema(DATASET_SCHEMA)
    val_df = load_data(DATASET)
    print(f"Number of data samples {val_df.shape[0]}")
    CODEX = []
    count = 0
    for index, row in val_df.iterrows():
        print("")
        print(f"index is {index}")
        print(row['query'])
        print(row['question'])
        table_list = None
        error_encountered = False
        error=None
        while table_list is None:
            try:
                prompt = schema_linking_prompt_maker(row['question'], row['db_id'])
                #print(prompt)
                table_list = local_generation(prompt)
            except Exception as e:
                # Handling other exceptions
                error = e
                error_encountered = True
                pass
        if not error_encountered:
            try:
                table_list = table_list.split("Tables_Required: ")[1]
            except Exception as e:
                error = e
                error_encountered = True
                table_list = "[]"
            
        fields = find_fields_MYSQL_like(row['db_id'])
        foreign_keys = "Foreign_keys = " + find_foreign_keys_MYSQL_like(row['db_id']) + '\n'
        schema = fields + foreign_keys
        
        CODEX.append({'question':row['question'],'gold_query':row['query'], 'db_id': row['db_id'], 'schema':schema, 'table_list':table_list, 'error_encountered':error_encountered, 'error_type': type(error).__name__, 'error_message':str(error)})


    OUTPUT_FILE_NAME = 'all-tables-mixtral.json'

    OUTPUT_PATH = os.path.join(OUTPUT_FILE, OUTPUT_FILE_NAME)

    file_path = "output.json"

    # Save the object to a JSON file
    with open(OUTPUT_PATH, 'w') as json_file:
        json.dump(CODEX, json_file)

Number of data samples 1034

index is 0
SELECT count(*) FROM singer
How many singers do we have?


Llama.generate: prefix-match hit


 In the question "How many singers do we have?", we are asked:
"singers" so we need table = [Singer] since we need column = [Singer.*]
Based on the tables, columns. So the Tables_required are:
Tables_Required: ['Singer']
index is 1
SELECT count(*) FROM singer
What is the total number of singers?



llama_print_timings:        load time =     789.79 ms
llama_print_timings:      sample time =      30.48 ms /    67 runs   (    0.45 ms per token,  2198.45 tokens per second)
llama_print_timings: prompt eval time =    1384.26 ms /    39 tokens (   35.49 ms per token,    28.17 tokens per second)
llama_print_timings:        eval time =    3185.31 ms /    66 runs   (   48.26 ms per token,    20.72 tokens per second)
llama_print_timings:       total time =    4839.68 ms /   105 tokens
Llama.generate: prefix-match hit


 In the question "What is the total number of singers?", we are asked:
"total number of singers" so we need table = [Singer] since we need column = [Singer.*]
Based on the tables, columns. So the Tables_required are:
Tables_Required: ['Singer']
index is 2
SELECT name ,  country ,  age FROM singer ORDER BY age DESC
Show name, country, age for all singers ordered by age from the oldest to the youngest.


...

 In the question "What region does Angola belong to and what is its population?", we are asked:
"region does Angola belong to" so we need table = [country] since we need column = [country.Region]
"population" so we need table = [country] since we need column = [country.Population]
Based on the tables, columns. So the Tables_required are:
Tables_Required: ['country']
index is 718
SELECT avg(LifeExpectancy) FROM country WHERE Region  =  "Central Africa"
What is the average expected life expectancy for countries in the region of Central Africa?



llama_print_timings:        load time =     789.79 ms
llama_print_timings:      sample time =      42.00 ms /    98 runs   (    0.43 ms per token,  2333.56 tokens per second)
llama_print_timings: prompt eval time =     823.09 ms /    23 tokens (   35.79 ms per token,    27.94 tokens per second)
llama_print_timings:        eval time =    4715.03 ms /    97 runs   (   48.61 ms per token,    20.57 tokens per second)
llama_print_timings:       total time =    5937.26 ms /   120 tokens
Llama.generate: prefix-match hit


 In the question "What is the average expected life expectancy for countries in the region of Central Africa?", we are asked:
"What is the average expected life expectancy" so we need table = [country] since we need column = [country.LifeExpectancy]
"for countries in the region of Central Africa" so we need table = [country] since we need column = [country.Region]
Based on the tables, columns. So the Tables_required are:
Tables_Required: ['country']
index is 719
SELECT avg(LifeExpectancy) FROM country WHERE Region  =  "Central Africa"
How long is the people’s average life expectancy in Central Africa?



llama_print_timings:        load time =     789.79 ms
llama_print_timings:      sample time =      47.49 ms /   111 runs   (    0.43 ms per token,  2337.14 tokens per second)
llama_print_timings: prompt eval time =     965.71 ms /    27 tokens (   35.77 ms per token,    27.96 tokens per second)
llama_print_timings:        eval time =    5349.78 ms /   110 runs   (   48.63 ms per token,    20.56 tokens per second)
llama_print_timings:       total time =    6773.34 ms /   137 tokens
Llama.generate: prefix-match hit


 In the question "How long is the people’s average life expectancy in Central Africa?", we are